<a href="https://colab.research.google.com/github/gvsnikhilreddy/s3-automation/blob/main/s3_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
%pip install spotipy
!pip install boto3

In [41]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import boto3
from datetime import datetime
from io import StringIO

In [42]:
client_credentials_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)

In [43]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [44]:
data = sp.playlist_tracks("3cEYpjA9oz9GiPac4AsH4n")

In [45]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    album_element = {'album_id': album_id,'album_name': album_name, 'release_date':album_release_date, 'total_tracks':album_total_tracks, 'url':album_url  }
    album_list.append(album_element)


In [46]:
album_df = pd.DataFrame.from_dict(album_list)

In [47]:
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
album_df['Timestamp'] = timestamp

# Generate a timestamped file name
file_name = f'album_data_{datetime.now().strftime("%Y%m%d%H%M%S")}.csv'

# Convert the DataFrame to CSV in memory
csv_buffer = StringIO()
album_df.to_csv(csv_buffer, index=False)

In [48]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,         # Replace with your AWS Access Key ID
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY # Replace with your AWS Secret Access Key
)

In [49]:
bucket_name = 'git-gitactions-s3'

In [50]:
try:
    s3_client.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())
    print(f"File uploaded successfully: s3://{bucket_name}/{file_name}")
except Exception as e:
    print(f"Error uploading file: {e}")

File uploaded successfully: s3://git-gitactions-s3/album_data_20241209144629.csv


# New section